In [1]:
import numpy as np      
import trimesh           
import pyglet
import scipy 
from scipy.ndimage import label
import matplotlib.pyplot as plt
from matplotlib.patches import Polygon
from matplotlib.collections import PatchCollection

#In the following cells we first define several functions needed for the (interactive) analysis of neck diameters & membrane distance

In [27]:
#Funct. 1: read in .tsi files
def read_sections(file_path):
    section_delimiter = "   "  # Delimiter of five spaces
    
    # Initialize lists for each section
    section1 = []
    section2 = []
    section3 = []
    
    # Flag to indicate the current section being read
    current_section = None
    
    # Read the file
    with open(file_path, 'r') as file:
        for line in file:
            line = line.strip()
            
            # Identify the section based on the starting word
            if line.startswith("vertex"):
                current_section = section1
                print(line)
                continue
            elif line.startswith("triangle"):
                current_section = section2
                print(line)
                continue
            elif line.startswith("inclusion"):
                current_section = section3
                print(line)
                continue
            
            # Skip the lines that mark the beginning of each section
            if current_section is None:
                continue
            
            # Split the line using the section delimiter
            if section_delimiter in line:
                entries = line.split(section_delimiter)
                # Remove empty entries (caused by consecutive delimiters)
                entries = [entry.strip() for entry in entries if entry.strip()]
                # Discard the first entry of each section
                if entries:
                    entries.pop(0)
                    # Convert entries based on the current section
                    if current_section is section1:
                        entries = [float(entry) for entry in entries]
                    elif current_section is section2:
                        entries = [int(entry) for entry in entries]
                    elif current_section is section3:
                        entries = [int(entry) if idx < len(entries) - 2 else float(entry) for idx, entry in enumerate(entries)]

                current_section.append(entries)
    
    return section1, section2, section3

In [28]:
#Funct. 2: read out the box size from .tsi files
def read_box(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()        
        second_line = lines[1].strip().split()
        # Check if the second line starts with 'box'
        if second_line[0] != 'box':
            raise ValueError("The second line does not start with 'box'.")
        # Convert the remaining elements to floats
        box = [float(value) for value in second_line[1:]]
        return box

In [29]:
#Funct. 3: correct a membrane mesh for periodic boundary conditions (move everything inside the box, remove triangles crossing the boundary)
def vis_mesh(mesh,box):
    new_triangles=[]
    #new_vertices=[] #use if the mesh splits in z-direction
    mesh1=mesh.faces
    for face in mesh1:
        p1,p2,p3=mesh.vertices[face[0]],mesh.vertices[face[1]],mesh.vertices[face[2]]
        if p1[2]<30:
            p1[2]=p1[2]+box[2]
        if p2[2]<30:
            p2[2]=p2[2]+box[2]
        if p3[2]<30:
            p3[2]=p3[2]+box[2]
            
        d_p12=np.linalg.norm(np.subtract(p1,p2))
        d_p13=np.linalg.norm(np.subtract(p1,p3))
        if d_p12<box[0]/2 and d_p13<box[0]/2:
            new_triangles.append(face)

    mesh_new=trimesh.Trimesh(vertices=mesh.vertices,faces=new_triangles)
    return mesh_new

In [30]:
#Funct. 4: Extension of Funct. 3, move a membrane mesh in the box, to center the neck for easy detection
#(in case of the neck e.g. being on the boundary, half of it will be on one side of the box half on the other)
def move_xy(mesh,box,direction,xshift=0.35,yshift=0.65):
    new_triangles,new_vertices=[],[] 
    mesh1=mesh.faces
    for face in mesh1[:]:#if the mesh splits in z-direction
        p1,p2,p3=mesh.vertices[face[0]],mesh.vertices[face[1]],mesh.vertices[face[2]]
        if p1[2]<30:
            p1[2]=p1[2]+box[2]
        if p2[2]<30:
            p2[2]=p2[2]+box[2]
        if p3[2]<30:
            p3[2]=p3[2]+box[2]

        #move by 25% boxsize in x or y-direction to the left, add that part to the right
        if direction=="x":
            a,b=0,xshift
            if p1[a]-b*box[a]<0:
                p1[a]=p1[a]+box[a]
            if p2[a]-b*box[a]<0:
                p2[a]=p2[a]+box[a]
            if p3[a]-b*box[a]<0:
                p3[a]=p3[a]+box[a]
        if direction=="y":
            a,b=1,yshift
            if p1[a]-b*box[a]<0:
                p1[a]=p1[a]+box[a]
            if p2[a]-b*box[a]<0:
                p2[a]=p2[a]+box[a]
            if p3[a]-b*box[a]<0:
                p3[a]=p3[a]+box[a]

        if direction=="xy":
            a,b=0,xshift
            if p1[a]-b*box[a]<0:
                p1[a]=p1[a]+box[a]
            if p2[a]-b*box[a]<0:
                p2[a]=p2[a]+box[a]
            if p3[a]-b*box[a]<0:
                p3[a]=p3[a]+box[a]

            c,d=1,yshift
            if p1[c]-d*box[c]<0:
                p1[c]=p1[c]+box[c]
            if p2[c]-d*box[c]<0:
                p2[c]=p2[c]+box[c]
            if p3[c]-d*box[c]<0:
                p3[c]=p3[c]+box[c]

        #removing triangles crossing the NEW=OLD boundary
        d_p12=np.linalg.norm(np.subtract(p1,p2))
        d_p13=np.linalg.norm(np.subtract(p1,p3))
        if d_p12<box[0]/2 and d_p13<box[0]/2:
            new_triangles.append(face)

    mesh_new=trimesh.Trimesh(vertices=mesh.vertices,faces=new_triangles)
    return mesh_new

In [31]:
#Funct. 5: divide the mesh into upper/lower (inner/outer) membrane, calc. their average distance
def inner_outer(mesh):
    triangles, vertices = mesh.triangles, mesh.vertices
    UPPER,LOWER= [],[]# Initialize list for z-coordinate of the centroid of triangles assigned to upper/lower membrane
    max_z,min_z=np.max(vertices[:,2]),np.min(vertices[:,2])
    dz=np.abs(max_z-min_z)
    
    for i in range(len(triangles)):
        X_t1 = np.mean(triangles[i], axis=0) #centroid
        if X_t1[2]<min_z+0.15*dz: #if centroid z-value is lower than 15% the maximal z-difference of the mesh above the lowest vertex of the mesh
            LOWER.append(X_t1[2])
        if X_t1[2]>max_z-0.15*dz: #if centroid z-value is higher than 15% the maximal z-difference of the mesh below the highest vertex of the mesh
            UPPER.append(X_t1[2])
            
    UPPER,LOWER=np.array(UPPER),np.array(LOWER)
    z_UPPER,z_LOWER=np.mean(UPPER),np.mean(LOWER)
    dz_mean=(z_UPPER+z_LOWER)/2
    dist_z=np.abs(z_UPPER-z_LOWER)
  
    return dist_z

In [32]:
#Funct. 5: project the mesh onto the XY-plane, and make a 2d binary image, so that membrane is black, holes are white
def plot_and_save_binary_image(visualize,mesh, box_size,resolution=1000):
    fig, ax = plt.subplots(figsize=(5, 5))

    # Plot the projected 2D mesh using Polygon
    for face in mesh.faces:
        vertices = mesh.vertices[face]
        polygon = Polygon(vertices[:, :2], closed=True, edgecolor='black', facecolor='black')
        ax.add_patch(polygon)

    vertices = mesh.vertices
    x_min,y_min,x_max,y_max=np.min(vertices[:,0]),np.min(vertices[:,1]),np.max(vertices[:,0]),np.max(vertices[:,1])
    ax.set_xlim(x_min+1,x_max-1)
    ax.set_ylim(y_min+1,y_max-1)
    

    # Remove axes for a cleaner look
    plt.axis('off')
    if visualize=="yes":
        plt.show()

    # Save the plot as a binary image
   # fig.canvas.draw()
   # image = np.frombuffer(fig.canvas.tostring_rgb(), dtype=np.uint8) #RGB image
   # image = image.reshape(fig.canvas.get_width_height()[::-1] + (3,)) #RGB image
   # gray_image = np.mean(image, axis=2) #grayscale
    fig.canvas.draw()
    image = np.asarray(fig.canvas.buffer_rgba())
    rgb_image = image[:, :, :3]  # Drop alpha channel
    gray_image = np.mean(rgb_image, axis=2)
    # Threshold to get binary image
    binary_image = gray_image < 128  # Assume triangles are black(1) and background is white(0)
    plt.close(fig)
    return binary_image


In [33]:
#Funct. 6: detect holes from the binary 2d projection of the mesh
def detect_holes(binary_image):
    # Invert the image, so holes are labeled as objects (1s become 0s, and 0s become 1s)
    inverted_image=np.invert(binary_image.astype(bool))
    labeled_holes, num_holes=label(inverted_image)
    #print(f"Detected {num_holes} holes.")
    return labeled_holes, num_holes

In [34]:
#Funct. 7: calculate hole size=neck diameter and position in pixels
def positions_and_sizes(labeled_holes, num_holes):
    hole_positions,hole_sizes=[],[]

    for i in range(1, num_holes + 1):
        # Find the pixels that belong to the current hole
        hole_pixels=np.argwhere(labeled_holes==i)
        
        # Calculate the average position of the hole
        centroid = np.mean(hole_pixels, axis=0)
        hole_positions.append(centroid)

        # Get the number of pixels in the hole
        hole_size = len(hole_pixels)
        hole_sizes.append(hole_size)

    return hole_positions, hole_sizes

In [35]:
#Funct. 8: convert diameter in pixels to diameter in l_DTS (unit of simulation mesh)
def pixels_to_diameter(num_pixels, box):
    # Calculate physical area from pixel count and size per pixel 
    pixel_size=box[0]/500 #binary_image.shape[0]
    num_pixels=np.array(num_pixels)
    area=num_pixels*(pixel_size**2)  
    # assuming circular approximation:
    diameter=2*np.sqrt(area/np.pi)
    return diameter

In [36]:
#Funct. 9: bring it all together:
#read in a .tsi file, make a mesh, correct the boundaries, shift the mesh if instructed, make 2d binary projection, detect neck & diameter and return them
def pore_detection(file_path, genus, shift, visualize,xshift=0.35,yshift=0.65): #shift="x","y","xy" or "". visualize="yes" or "no"
    vertex, triangle, inclusion= read_sections(file_path) #reads in and prints header of each section
    mesh = trimesh.Trimesh(vertices=vertex,faces=triangle) #makes original mesh
    box=read_box(file_path) #reads box size
    d_z=inner_outer(mesh) #average distance between upper & lower membrane

    if shift=="":
        NewMesh=vis_mesh(mesh,box)
    else:
        NewMesh=move_xy(mesh,box,shift,xshift,yshift)

    #project to 2d, make binary image, detect features (black=background-block, white=holes=pores)
    binary_image = plot_and_save_binary_image(visualize,NewMesh, box_size=box[:2]) 
    labeled_holes, num_holes = detect_holes(binary_image)
    hole_positions, hole_sizes =positions_and_sizes(labeled_holes, num_holes)
    hole_positions,hole_sizes= hole_positions[:],hole_sizes[:] #delete the first feature, as is it always the background

    # Output the results
    #for i in range(num_holes):
    #    print(f"Hole {i+1}: Position = {hole_positions[i]}, Size = {hole_sizes[i]}")
    return num_holes,hole_positions,hole_sizes,d_z

In [ ]:
#INTERACTIVE ANALYSIS
#Specify the frames to analyse (for filename):
TSI = np.array([1,6,15,22,40,55,60,68,71,76,79,81,83,86,91,94,96,98,101,103,106,108,110,112,114,117,120,121,124,126,130,133,136,138,141,143,145,148,150]) 
kappa=np.array([5,10,20]) #bending rigidity (also for filename)
nrep=10 #number of replicas
ALLpositions,ALLsizes,ALLmem_dist=[],[],[] #lists to collect results for all frames at all bending rigidities and all replica
for i in range(0,len(TSI)):
    TSIpositions,TSIsizes,TSImem_dist=[],[],[]
    print("TSI{}".format(TSI[i]))
    for j in range(0,len(kappa)):
        Kappapositions,Kappasizes,Kappamem_dist=[],[],[]
        print("Kappa{}".format(kappa[j]))
        for k in range(1,11): #loop through replicas
            file_path="{}TSI_Kappa{}/rep{}/TrjTSI/output395.tsi".format(TSI[i],kappa[j],k)
            shifts=[0.0,0.0]
            box=read_box(file_path)
            while True: #try to detect the neck; if it can't be found after trying usual shifts (bc. it is split in a complicated way over the box):
                #prompt user input for shifting the mesh to center the neck
                num_holes,hole_positions,hole_sizes,d_z=pore_detection(file_path, genus=1, shift="xy",xshift=shifts[0],yshift=shifts[1],visualize="no")
                if num_holes==2:
                    diameter=pixels_to_diameter(hole_sizes[1:], box)
                    Kappasizes.append(diameter)
                    Kappapositions.append(hole_positions[1:])
                    Kappamem_dist.append(d_z)
                    print("positions and sizes appended for", file_path,diameter,d_z)
                    break
                else:
                    shifts1,shifts2,shifts3,shifts4=[0.35,0.0],[0.0,0.65],[0.35,0.65],[0.15,0.85]
                    num_holes1,hole_positions1,hole_sizes1,d_z1=pore_detection(file_path, genus=1, shift="xy",xshift=shifts1[0],yshift=shifts1[1],visualize="yes")
                    num_holes2,hole_positions2,hole_sizes2,d_z2=pore_detection(file_path, genus=1, shift="xy",xshift=shifts2[0],yshift=shifts2[1],visualize="no")
                    num_holes3,hole_positions3,hole_sizes3,d_z3=pore_detection(file_path, genus=1, shift="xy",xshift=shifts3[0],yshift=shifts3[1],visualize="no")
                    num_holes4,hole_positions4,hole_sizes4,d_z4=pore_detection(file_path, genus=1, shift="xy",xshift=shifts4[0],yshift=shifts4[1],visualize="no")
                    if num_holes1==2:
                        diameter=pixels_to_diameter(hole_sizes1[1:], box)
                        Kappasizes.append(diameter)
                        Kappapositions.append(hole_positions1[1:])
                        Kappamem_dist.append(d_z1)
                        print("positions and sizes appended for", file_path,diameter)
                        break
                    elif num_holes2==2:
                        diameter=pixels_to_diameter(hole_sizes2[1:], box)
                        Kappasizes.append(diameter)
                        Kappapositions.append(hole_positions2[1:])
                        Kappamem_dist.append(d_z2)
                        print("positions and sizes appended for", file_path,diameter)
                        break
                    elif num_holes3==2:
                        diameter=pixels_to_diameter(hole_sizes3[1:], box)
                        Kappasizes.append(diameter)
                        Kappapositions.append(hole_positions3[1:])
                        Kappamem_dist.append(d_z3)
                        print("positions and sizes appended for", file_path,diameter)
                        break

                    elif num_holes4==2:
                        diameter=pixels_to_diameter(hole_sizes4[1:], box)
                        Kappasizes.append(diameter)
                        Kappapositions.append(hole_positions4[1:])
                        Kappamem_dist.append(d_z4)
                        print("positions and sizes appended for", file_path,diameter)
                        break

                    else:
                        print(f"For file '{file_path}': num_holes = {num_holes} is 0.")
                        inputs = input("Enter a new value for shift: ")
                        value1,value2,hand=inputs.split(',')
                        shifts,hand=[float(value1),float(value2)],[float(hand)]
                        if hand[0]!=0.0:
                            Kappasizes.append(hand)
                            Kappapositions.append(hole_positions[0])
                            Kappamem_dist.append(d_z)
                            break
                    
            print("here Kappa appended")
        TSIpositions.append(Kappapositions)
        TSIsizes.append(Kappasizes)
        TSImem_dist.append(Kappamem_dist)
        print("here TSI appended")
    ALLpositions.append(TSIpositions)
    ALLsizes.append(TSIsizes)
    ALLmem_dist.append(TSImem_dist)
    print("here ALL appended")
ALLpositions = np.array(ALLpositions,dtype=object)
ALLsizes = np.array(ALLsizes,dtype=object)
ALLmem_dist = np.array(ALLmem_dist,dtype=object)
print("here")

In [14]:
#Save the results
np.save('ALLsizesGen1Aconst-inter.npy',ALLsizes)
np.save('ALLmemDistGen1Aconst-inter.npy',ALLmem_dist)
np.save('ALLpositionsGen1Aconst-inter.npy', ALLpositions)